# Chain level Steenrod operations

### Abstract
In this notebook we implement functions that construct representatives for Steenrod operations acting on algebras over the Barratt-Eccles and Surjection $E_\infty$-operads. An important example we treat in detail is the case of cochains of simplicial sets. 

## Introduction
...

## Preliminaries

### Free modules

**Definition**. Let $R$ be a ring and $B$ a set. The free $R$-module generated by $B$ consists of all $R$-linear combination of elements in $B$

\begin{equation*}
R[B] = \Big\{ \sum_i r_ib_i\ |\ r_i \in R, b_i \in B \Big\}.
\end{equation*}

For a ring $R$ equal to $\mathbb Z$ or $\mathbb Z/n\mathbb Z$ we introduce a class modeling elements in free $R$-modules. The ring is specified via the class attribute `torsion`, where `'free'` sets $R = \mathbb Z$ and a positive `int` sets $R = \mathbb Z/n\mathbb Z$. The class supports `+` addition, `-` substraction and `*` scaling.

In [1]:
from clesto import Module_element

# The initialization takes a dict with int values
x = Module_element({'a':1, 'b':2, 'c':3})

print(f'If x = {x} then \n' +
      f'-x = {-x} \n' + 
      f'x+x = {x+x} \n' + 
      f'2*x = {2*x} \n' + 
      f'x-x = {x-x}')

b1 = 4((1,2,3),(1,3,2))+5((3,1,2),(3,2,1))
b2 = 3((2,1,3),(2,3,1))
b1 o_2 b2 = -12((1,3,2,4,5),(1,3,4,2,5),(1,5,3,4,2))+12((1,3,2,4,5),(1,5,3,2,4),(1,5,3,4,2))-15((5,1,3,2,4),(5,1,3,4,2),(5,3,4,2,1))+15((5,1,3,2,4),(5,3,2,4,1),(5,3,4,2,1))
If x = a+2b+3c then 
-x = -a-2b-3c 
x+x = 2a+4b+6c 
2*x = 2a+4b+6c 
x-x = 0


In [2]:
#_________________________________79_characters________________________________

# The default value of the attribute torsion is 'free'. 
# It can be set in three ways:
# a) For all future instances using the class attribute default_torsion
# b) Using the flag torsion at initialization
# c) Using the setter method set_torsion

Module_element.default_torsion = 'free'
x = Module_element({'a':1, 'b':2, 'c':3})
print(f'x = {x} with torsion = {x.torsion}')
y = Module_element(x, torsion=3)
print(f'x = {y} with torsion = {y.torsion}')
x.set_torsion(2)
print(f'x = {x} with torsion = {x.torsion}')

x = a+2b+3c with torsion = free
x = a+2b with torsion = 3
x = a+c with torsion = 2


### Group rings

**Definition**. Let $R$ be a ring and $\Gamma$ a group. The free $R$-module generated by $\Gamma$ is a ring with the induced product

\begin{equation*}
\left( \sum_i r_ia_i \right) \left( \sum_j s_jb_j \right) = \sum_{i,j} (r_is_j)(a_ib_{j}).
\end{equation*}

#### Cyclic groups

Let $C = \langle \rho \rangle$ be the infinite cyclic group and $C_r = \langle \rho\ |\ \rho^r = 1 \rangle$ be the cyclic group of order $r$.

We introduce a subclass of `Module_element` modeling elements in the group ring of cyclic groups when the base ring is $\mathbb Z$ or $\mathbb Z/n\mathbb Z$. It inherents the `torsion` attribute, as well as the addition `+`, substraction `-`, and scaling `*` methods; additionally, it has an `order` attribute and supports the ring product `*`.

In [3]:
from clesto import Cyclic_Module_element

# Default order is 'infinite'.
x = Cyclic_Module_element({0:1, 1:2, 2:3, 3:4}, order=3, torsion=2)
print(f'The order of x = {x} was set to {x.order} and x*x = {x*x}')

The order of x = 1+q^2 was set to 3 and x*x = 1+q


There are two important elements in $\mathbb Z/n\mathbb Z[C_n]$

\begin{equation*}
T = \rho-1 \quad \text{ and } \quad N = 1+\rho+\cdots+\rho^{n-1}
\end{equation*}

defining a minimal resolution

\begin{equation*}
W: \qquad \mathbb Z/n\mathbb Z[C_n] \stackrel{T}{\longleftarrow} \mathbb Z/n\mathbb Z[C_n] \stackrel{N}{\longleftarrow} \mathbb Z/n\mathbb Z[C_n] \stackrel{T}{\longleftarrow} \cdots
\end{equation*}

of $\mathbb Z/n\mathbb Z$ by a free differential graded $\mathbb Z/n\mathbb Z[C_n]$-module. We denote a chosen generator in degree $d$ by $e_d$.

The class `Cyclic_Module_element` contains the methods `transposition_element` and `norm_element` returning models for $T$ and $N$ given an `int`. 

In [4]:
n = 4
T = Cyclic_Module_element.transposition_element(n)
N = Cyclic_Module_element.norm_element(n)
print(f'For Z/{n}Z[C_{n}] the special elements are \n' +
      f'T = {T} and N = {N}. \n' +
      f'They satisfy T*N = {T*N} and N*T = {N*T}')

For Z/4Z[C_4] the special elements are 
T = q+3 and N = 1+q+q^2+q^3. 
They satisfy T*N = 0 and N*T = 0


#### Symmetric groups

Let $\Sigma_r$ denote the group of permutations of $r$ elements. More explicitly, a permutation is a bijection of set $\{1,\dots,r\}$ and $r$ is referred to as the arity of the permutation. A permutation $\pi$ can be represented by the tuple of its images $(\pi(1), \dots, \pi(r))$.

We introduce a subclass of `Module_element` modeling elements in $\mathbb Z[\Sigma_r]$ and $\mathbb Z/n\mathbb Z[\Sigma_r]$. It inherents addition `+`, substraction `-`, and scaling `*` and, additionally, it supports the ring product `*`.

In [5]:
from clesto import Symmetric_Module_element

x = Symmetric_Module_element({(2,3,1):5}, torsion=3)
print(f'If x = {x} its arity is {x.arity} and \nx*x = {x*x} and x*x*x = {x*x*x}')

If x = 2(2,3,1) its arity is 3 and 
x*x = (3,1,2) and x*x*x = 2(1,2,3)


#### Composition (TO DO: description)

For all integers $r, s \geq 1$ and $k \in \{1, \ldots, s\}$ we define a partial composition
\begin{equation*}
\circ_k : \Sigma_r \times \Sigma_s \to \Sigma_{r+s-1} \, .
\end{equation*}

In [6]:
from clesto import Symmetric_Module_element

x = Symmetric_Module_element({(3,2,1):1})
y = Symmetric_Module_element({(1,3,2):1})
print(f'If x = {x} and y = {y}\nthen x o_2 y = {x.compose(y,2)}')

If x = (3,2,1) and y = (1,3,2)
then x o_2 y = (5,2,4,3,1)


We also define a composition
\begin{equation*}
	\circ_{\Sigma} : \Sigma_r \times \Sigma_{s_1} \times \cdots \times \Sigma_{s_r} \ \to \
	\Sigma_{s_1 + \cdots + s_{r}} \, .
\end{equation*}

In [7]:
from clesto import Symmetric_Module_element

x = Symmetric_Module_element({(3,2,1):1})
y1 = Symmetric_Module_element({(1,2):1})
y2 = Symmetric_Module_element({(1,3,4,5,2):1})
y3 = Symmetric_Module_element({(4,1,3,2):1})
print(f'x = {x}\ny1 = {y1}\ny2 = {y2}\ny3 = {y3}\nx o (y1,y2,y3) = {x.compose(y1,y2,y3)}')

x = (3,2,1)
y1 = (1,2)
y2 = (1,3,4,5,2)
y3 = (4,1,3,2)
x o (y1,y2,y3) = (11,8,10,9,3,5,6,7,4,1,2)


### Coskeletal differential graded $R$-modules 

**Definition**. Let $R$ be a ring and $B$ a set. The coskeletal differential graded $R$-module generated by $B$ is the differential graded $R$-module

\begin{equation*}
N_d(B;R) = \frac{R \big\{ (b_0,\dots,b_d)\ |\ b_i \in B \big\}}{R \{b_i = b_{i+1} \}}
\qquad
\partial(b_0,\dots,b_d) = \sum_{i=1}^n (-1)^i(b_0,\dots,\widehat{b}_i,\dots,b_d).
\end{equation*}

**Remark 1**. This terminology is motivated by the fact that $N_\bullet(B;R)$ is isomorphic to the normalized chains of the 0-coskeletal simplicial set generated by $B$.

We introduce a subclass of `Module_element` that models elements in $N_\bullet(B;\mathbb Z)$ and $N_\bullet(B;\mathbb Z/n\mathbb Z)$. It inherents addition `+`, substraction `-`, and scaling `*`, additionally, it contains a method `boundary` modeling $\partial$.

In [8]:
from clesto import DGModule_element

# It automatically removes degenerate simplices
x = DGModule_element({('a','b','c'):1, ('a','a','c'):2, ('a','b','a'):2})
print(f'Let x = {x} then\nboundary(x) = {x.boundary()}')

Let x = ('a','b','c')+2('a','b','a') then
boundary(x) = ('b','c')-('a','c')+3('a','b')+2('b','a')


### Coskeletal differential graded $R[\Gamma]$-modules

Let $R$ be a ring and $\Gamma$ a group. The coskeletal differential graded $R$-module generated by $\Gamma$ is naturally a left $R[\Gamma]$-module via

\begin{equation*}
\gamma (\gamma_0, \dots, \gamma_d) = (\gamma \gamma_0, \dots, \gamma \gamma_d).
\end{equation*}

We introduce a subclass of `DGModule_element` that models elements in $N_\bullet(C;\mathbb Z)$ and $N_\bullet(C_n;\mathbb Z/m\mathbb Z)$. It inherents addition `+`, substraction `-`, scaling `*`, and the `boundary` method, additionally, it supports the action `*` of the group ring.

In [9]:
from clesto import Cyclic_DGModule_element

n = m = 3
x = Cyclic_DGModule_element({(0,1,0,4):1}, torsion=n, order=m)
N = Cyclic_Module_element.transposition_element(n)
print(f'Let x = {x} and N = {N} then \n' + 
      f'Nx = {N(x)} and \n' +
      f'boundary(x) = {x.boundary()}')

Let x = q^(0,1,0,1) and N = q+2 then 
Nx = q^(1,2,1,2)+2q^(0,1,0,1) and 
boundary(x) = q^(1,0,1)+2q^(0,1,0)


## Power Operations

**Definition**. Let $R$ be a ring. An operad $\mathcal O$ is a set $\big\{ \mathcal O(r) \big\}_{r \in \mathbb N}$ of differential graded left $R[\Sigma_r]$-modules and morphisms

\begin{equation*}
\circ_{\mathcal O} : \mathcal O(r) \otimes \mathcal O(s_1) \otimes \cdots \otimes \mathcal O(s_r) \to \mathcal O(s_1+\cdots+s_r)
\end{equation*}

satisfying: associativity, unitality and equivariance. The operad $\mathcal O$ is said to be an $E_\infty$-operad if $O(r)$ is a free $R[\Sigma_r]$-module resolution of $R$.

An algebra over an operad $\mathcal O$ is a collection of maps

\begin{equation*}
\mathcal O(r) \otimes A^{\otimes r} \to A
\end{equation*}

compatible with the operadic structure. This structure is equivalent to an operad map 

\begin{equation*}
\mathcal O \to \mathrm{End}_{A}
\end{equation*}

where the operad
\begin{equation*}
\mathrm{End}_{A} = \big\{ \mathrm{End}(A^{\otimes r}, A) \big\}_{r \geq 0}
\end{equation*}

has composition structure induced from the usual composition of linear maps and the group action is induced from permutations of factors in the domain.

**Definition** Let $p$ be a prime number and $R = \mathbb Z/ p \mathbb Z$. Let $\mathcal O$ be an $E_\infty$-operad and $A$ and $\mathcal O$-algebra. Choosing an $\mathbb F_p[\mathrm C_p]$-equivariant quasi-isomorphism

\begin{equation} \label{equation: from W to O}
\iota : W \to \mathcal O(p)
\end{equation}
let
\begin{equation*}
D_i : A \to A
\end{equation*}
be defined by
\begin{equation*}
D_i(a) = \iota(e_i)(a \otimes \cdots \otimes a).
\end{equation*}
If p = 2, define
\begin{equation*}
Q^s : A \to A
\end{equation*}
for $a \in A$ of degree $d$ by
\begin{equation*}
Q^s(a) = \begin{cases}
D_{s-q}(a) & s \geq d \\
0 & s < d
\end{cases}
\end{equation*}
and, if $p > 2$,
\begin{equation*}
Q^s : A \to A \qquad
\beta Q^s : A \to A
\end{equation*}
are respectively defined by
\begin{equation*}
Q^s(a) = \begin{cases}
D_{(2s-d)(p-1)}(a) & 2s \geq d \\
0 & 2s < d
\end{cases}
\qquad
\beta Q^s(a)  = \begin{cases}
D_{(2s-d)(p-1)-1}(a) & 2s > d \\
0 & 2s \leq q.
\end{cases}
\end{equation*}
The homology operations
\begin{equation*}
P^s : H_\bullet(A) \to H_{\bullet + 2s(p-1)}(A),
\end{equation*}

which are independent of $\iota$, are defined for a class $[a]$ of degree $d$ with a cocycle representative $a \in A$ by

\begin{align*}
P^s\big([a]\big) & = (-1)^s \nu(d) \big[Q^s(a)\big] \\
\beta P^s\big([a]\big) & = (-1)^s \nu(d) \big[\beta Q^s(a)\big]
\end{align*}
where $\nu(d) = (-1)^{d(d-1)m/2}(m!)^d$ and $m = (p-1)/2$.

This choice of coefficient is credited to Serre and it is motivated by the identity $D_{q(p-1)}(a) = \nu(d)\, a$.

### Barrat-Eccles operad $\mathcal E$

**Definition**. Let $R$ be a ring. The arity $r$ part $\mathcal E(r)$ of the Barratt-Eccles operad $\mathcal E$ is the coskeletal differential graded $R[\Sigma_r]$-module generated by $\Sigma_r$. Its composition is best described using Remark 1. 

We introduce a subclass of `DGModule_element` that models elements in the Barrat-Eccles operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*`, and `boundary` methods, additionally, it supports the left action of the correponding symmetric group ring `*`, and it contains the method `compose` modeling the operadic composition.

In [10]:
from clesto import Barratt_Eccles_element

b = Barratt_Eccles_element({((1,2,3),(3,2,1)):1})
x = Symmetric_Module_element({(1,3,2):1})

print(f'Let b = {b} and x = {x} then \n' + 
      f'x(b) = {x(b)} and \n' +
      f'boundary(b) = {b.boundary()}')

Let b = ((1,2,3),(3,2,1)) and x = (1,3,2) then 
x(b) = ((1,3,2),(2,3,1)) and 
boundary(b) = ((3,2,1),)-((1,2,3),)


#### Composition (TO DO: description)

For all integers $r, s, d, e \geq 1$ and every $k \in \{1, \ldots, s\}$ we define a partial composition
\begin{equation*}
\circ_k : \mathcal E(r)_d \otimes \mathcal E(s)_e \to \mathcal E(r + s - 1)_{d+e} \, .
\end{equation*}

In [11]:
from clesto import Barratt_Eccles_element

b1 = Barratt_Eccles_element({((1,2,3),(1,3,2)):4, ((3,1,2),(3,2,1)):5})
b2 = Barratt_Eccles_element({((2,1,3),(2,3,1)):3})

print(b1.__dict__, b2.__dict__)
print(f'b1 = {b1}\nb2 = {b2}\nb1 o_2 b2 = {b1.compose(b2, 2)}')

{'arity': 3, 'torsion': 'free'} {'arity': 3, 'torsion': 'free'}
b1 = 4((1,2,3),(1,3,2))+5((3,1,2),(3,2,1))
b2 = 3((2,1,3),(2,3,1))
b1 o_2 b2 = -12((1,3,2,4,5),(1,3,4,2,5),(1,5,3,4,2))+12((1,3,2,4,5),(1,5,3,2,4),(1,5,3,4,2))-15((5,1,3,2,4),(5,1,3,4,2),(5,3,4,2,1))+15((5,1,3,2,4),(5,3,2,4,1),(5,3,4,2,1))


We also define a composition
\begin{equation*}
\circ : \mathcal E(r)_d \otimes \mathcal E(s_1)_d \otimes \cdots \otimes \mathcal E(s_r)_d \to \mathcal E(s_1 + \cdots + s_r)_{(r+1)d} \, .
\end{equation*}

In [12]:
x = Module_element({'a':1})
y = x
y += y
print(x)

2a


In [32]:
from clesto import Barratt_Eccles_element

b = Barratt_Eccles_element({((1,2,3),(1,3,2)):1})
c1 = Barratt_Eccles_element({((2,1),(1,2)):1})
c2 = Barratt_Eccles_element({((1,2,3),):2})
c3 = Barratt_Eccles_element({((2,1),):3})

print(f'For b  = {b} \n' +
      f'c1 = {c1} \n' +
      f'c2 = {c2} \n' + 
      f'c3 = {c3} \nwe have\n' +
      f'b o (c1,c2,c3) = {b.compose(c1,c2,c3)}')

For b  = ((1,2,3),(1,3,2)) 
c1 = ((2,1),(1,2)) 
c2 = 2((1,2,3),) 
c3 = 3((2,1),) 
we have
b o (c1,c2,c3) = -6((2,1,3,4,5,7,6),(1,2,3,4,5,7,6),(1,2,7,6,3,4,5))+6((2,1,3,4,5,7,6),(2,1,7,6,3,4,5),(1,2,7,6,3,4,5))


We introduced a canonical choice for
\begin{equation*}
\iota : W \to \mathcal E(p)
\end{equation*}

and use it to define explicit operations $P^s$ and $\beta P^s$ for Barratt-Eccles algebras. This map factors as

\begin{equation*}
W \stackrel{\psi}{\to} N_\bullet(\mathrm C_p; \mathbb Z/ p \mathbb Z) \stackrel{\phi}{\to} \mathcal E(p).
\end{equation*}
    
#### The map $\psi$

The quasi-isomorphism $\psi : W \to N_\bullet(C_p;\mathbb Z/p\mathbb Z)$ is defined by

\begin{align*}
\psi(e_{2k}) & = 
\sum_{i=1}^{k} \sum_{r_i=1}^{p-1}
\big( 1, \rho^{I_1(r_1,\dots,r_k)}, \dots,\ \rho^{I_{2k}(r_1,\dots,r_k)} \big) \\
\psi(e_{2k+1}) & = 
\sum_{i=1}^{k} \sum_{r_i=1}^{p-1}
\big( 1, \rho, \rho^{I_1(r_1,\dots,r_k)+1}, \dots,\ \rho^{I_{2k}(r_1,\dots,r_k)+1} \big)
\end{align*}
where
\begin{equation*}
I_s(r_1,\dots,r_k) =
r_1 + \cdots + r_{\lceil s/2 \rceil} + \lfloor s/2 \rfloor
\end{equation*}

with $\lceil - \rceil$ and $\lfloor - \rfloor$ denoting the integer ceiling and floor functions.

The class `Cyclic_Module_element` contains a method `psi` modeling this map.

In [36]:
n = 3
N = Cyclic_Module_element.norm_element(3)

print(f'Let us consider the norm element for Z/{n}Z[C_{n}]\n'+
      f'in degree 2: N = {N} then \n'+
      f'psi(N) = {N.psi(2)}')

Let us consider the norm element for Z/3Z[C_3]
in degree 2: N = 1+q+q^2 then 
psi(N) = q^(0,1,2)+q^(1,2,0)+q^(2,0,1)+q^(0,2,0)+q^(1,0,1)+q^(2,1,2)


#### The map $\phi$

The generator $\rho$ of $C_p$ can be regaded as a permutation by 

\begin{equation*}
\rho(1,2,\dots,p) = (2,\dots,p,1)
\end{equation*}

which induces an inclusion of associated coskeletal differential graded modules

\begin{equation*}
\phi: N_\bullet(C_p;\mathbb Z/p\mathbb Z) \to N_\bullet(\Sigma_p;\mathbb Z/p\mathbb Z) = \mathcal E(p)
\end{equation*}

The class `Cyclic_DGModule_element` contains a method `phi` modeling this map.

In [15]:
x = Cyclic_DGModule_element({(0,5,0,5):1}, torsion=4, order=4)

print(f'Let x = {x} then \nphi(x) = {x.phi()}')

Let x = q^(0,1,0,1) then 
phi(x) = ((1,2,3,4),(2,3,4,1),(1,2,3,4),(2,3,4,1))


### Surjection operad

**Definition** Let $R$ be a ring. The arity $r$ part $\mathcal Surj(r)$ of the Surjection operad $Surj$ is the coskeletal differential graded $R$-module generated by $\{1,\dots,r\}$ modulo tuples defining functions $\{1,\dots,r+d\} \to \{1,\dots,r\}$ that fail to be surjective. The symmetric group action is induced for any $\sigma \in \Sigma_r$ by

\begin{equation*}
\sigma (s(1), \dots, s(r+s)) = (\sigma s(1), \dots, \sigma s(r+s))
\end{equation*}

DESCRIBE COMPOSITION

We introduce a subclass of `DGModule_element` that models elements in the Surjection operad when $R = \mathbb Z$ or $R = \mathbb Z/n\mathbb Z$. It inherents addition `+`, substraction `-`, scaling `*`, and `boundary` method, additionally, it supports the left action of the correponding symmetric group ring `*`, and it contains a method `compose` modeling the operadic composition.

IMPLEMENT COMPOSITION

In [16]:
from clesto import Surjection_element

s = Surjection_element({(1,2,3,1):1, (2,3,2,1):2, (2,3,2,3):1})
x = Symmetric_Module_element({(2,1,3):2})

print(f'Let s = {s} \nand x = {x} then \nx(s) = {x(s)}')

Let s = (1,2,3,1)+2(2,3,2,1) 
and x = 2(2,1,3) then 
x(s) = 2(2,1,3,2)+4(1,3,1,2)


### $TR : \mathcal E \to Surj$

The table reduction morphism is a quasi-ismomorphism

\begin{equation*}
TR : \mathcal E \to Surj
\end{equation*}

whose definition is presented in [1].

The class `Barratt_Eccles_element` contains the method `table_reduction` modeling this map.

In [17]:
T = Cyclic_Module_element.transposition_element(3)
print(f'Let T be the transposition element {T} in degree 1\n'+
      f'psi(T) = {T.psi(1)} \n'+
      f'phi(psi(T)) = {T.psi(1).phi()}\n'+
      f'TR(phi(psi(T))) = {T.psi(1).phi().table_reduction()}')

Let T be the transposition element q+2 in degree 1
psi(T) = q^(1,2)+2q^(0,1) 
phi(psi(T)) = ((2,3,1),(3,1,2))+2((1,2,3),(2,3,1))
TR(phi(psi(T))) = (2,3,1,2)+2(1,2,3,1)


### Eilenberg-Zilber operad

**Definition** Operad $\mathrm{End}(N_\bullet)$ of natural transformations of the functor of normalized chains.

**Lemma** A basis given by pairs $(op, n)$ where $op = d_{U_1} \otimes \cdots \otimes d_{U_r}$ and $n$ is the degree of the domain. 

In [18]:
from clesto import Eilenberg_Zilber_element

x = Eilenberg_Zilber_element({((1,2),(2,1,2),tuple()):2,((1,2),(1,1,2),(1,)):3})
print(x)

2(d_1d_2)x(d_1d_2d_4)x(id) + 3(d_1d_2)x(d_1d_2d_3)x(d_1)


### $Surj \to \mathrm{End}(N_\bullet)$


A quasi-isomorphism described in $[1]$

MISSING DEFINITION

The class `Surjection_element` has a method `as_Eilenberg_Zilber_element` modeling this map 

MISSING SIGN

In [19]:
cup_3 = Surjection_element({(1,2,1,2,1):3}, torsion=2)
print(cup_3.to_Eilenber_Zilber_element(4))

(id)x(d_4) + (d_3)x(id) + (id)x(d_2) + (d_1)x(id) + (id)x(d_0)
